In [24]:
import tensorflow as tf
tf.enable_eager_execution()
import time
import pickle
from CleaningFunctions import *

In [25]:
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [26]:
# get all segment file names and save in Filenames
folder = '/media/arash/My Passport/waymo'#'./waymo-od/data'
keyword = 'segment'
Filenames=scan(folder,keyword)

In [27]:
len(Filenames)

125

In [28]:
### Extract segment files in Frames, each file consists of multiple frames

ObjectType = 2   #object codes, unknown:0, veh:1, ped:2, sign:3, bike:4   
count = len(Filenames)


for i in range(int(count/5)):     #read files three by three
    runtime0 = time.process_time()
    Frames = {}
    for file in Filenames[5*i:(5*i+5)]:
        SegID = getID(file)
        Segment = []
        dataset = tf.data.TFRecordDataset(file, compression_type='')
        for data in dataset:
            frame = open_dataset.Frame()
            frame.ParseFromString(bytearray(data.numpy()))
            Segment.append(frame)
        Frames[SegID] = Segment
        
    ObjData = MultiFrame(Frames,ObjectType)
    fileNumber = getID(file)[0:4]
    name = f"PedTables/File_{fileNumber}_Iterate_{5*i}:{5*i+5}.p"
    with open(name, 'wb') as fp:
        pickle.dump(ObjData, fp)
        
    runtime1 = time.process_time()
    print(f"File {i} Saved, Process Time: {runtime1 - runtime0}")
    del ObjData
    del Frames
  


File 0 Saved, Process Time: 22.072580308
File 1 Saved, Process Time: 16.135549958
File 2 Saved, Process Time: 15.904499625
File 3 Saved, Process Time: 14.816453932999998
File 4 Saved, Process Time: 15.41924674500001
File 5 Saved, Process Time: 17.49098483099999
File 6 Saved, Process Time: 16.666466060000005
File 7 Saved, Process Time: 16.719602324999983
File 8 Saved, Process Time: 16.190377693000016
File 9 Saved, Process Time: 16.499252916000017
File 10 Saved, Process Time: 18.014279559000016
File 11 Saved, Process Time: 17.213230353
File 12 Saved, Process Time: 14.454859559999989
File 13 Saved, Process Time: 16.427120650999996
File 14 Saved, Process Time: 15.37631891099997
File 15 Saved, Process Time: 15.534693058000016
File 16 Saved, Process Time: 17.115309222999997
File 17 Saved, Process Time: 16.07226275900001
File 18 Saved, Process Time: 15.981972059999975
File 19 Saved, Process Time: 15.10505738500001
File 20 Saved, Process Time: 16.59999338800003
File 21 Saved, Process Time: 17.

In [29]:
## combining all the dics together
# get all names pf dict created and save in Filenames
folder = './PedTables'#'./waymo-od/data'
keyword = 'File'
Filenames=scan(folder,keyword)

In [34]:
## Open and merge all
AllPeds = {}
for i in range(len(Filenames)):
    with open(Filenames[i], 'rb') as fp:
        AllPeds.update(pickle.load(fp))

In [49]:
with open('./PedTables/AllPeds.p', 'wb') as fp:
    pickle.dump(AllPeds, fp)

In [51]:
PedIDs = list(AllPeds.keys())
AllPeds[PedIDs[0]]

,Timestamp,x,y,z,px_GF,py_GF,pz_GF,x_GF_SDC,y_GF_SDC,z_GF_SDC,...,Speed x,Speed y,accel x,accel y,Veh Count,Ped Count,Bike Count,Weather,Time of Day,Segment ID
0,1.550081e+15,31.323749,6.672903,1.129911,-1553.566520,9891.559600,34.450858,-1545.183012,9922.485056,33.890,...,-1.097628,0.701208,-1.439460e-01,9.195842e-02,34.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
1,1.550081e+15,31.212542,6.531768,1.121214,-1553.682991,9891.634006,34.442858,-1545.227580,9922.396715,33.890,...,-1.112022,0.710404,-1.439460e-01,9.195842e-02,34.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
2,1.550081e+15,31.099077,6.388354,1.118753,-1553.799454,9891.708407,34.434858,-1545.274079,9922.306370,33.889,...,-1.126417,0.719600,-1.439460e-01,9.195842e-02,34.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
3,1.550081e+15,30.982323,6.245735,1.107891,-1553.915931,9891.782817,34.426858,-1545.322268,9922.213059,33.888,...,-1.140812,0.728796,-1.439460e-01,9.195842e-02,34.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
4,1.550081e+15,30.859727,6.104866,1.091610,-1554.032395,9891.857219,34.418858,-1545.371961,9922.114396,33.886,...,-1.155206,0.737992,-1.439460e-01,9.195842e-02,34.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,1.550081e+15,-2.089355,-9.777067,0.748899,-1568.152923,9899.452048,35.001971,-1560.689708,9892.819852,34.090,...,-1.072319,0.451414,-1.516815e-13,2.598795e-13,42.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
145,1.550081e+15,-2.533169,-9.905042,0.789499,-1568.260299,9899.497250,34.999749,-1560.944445,9892.363340,34.142,...,-1.072319,0.451414,-1.516815e-13,2.598795e-13,41.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
146,1.550081e+15,-2.972048,-10.040041,0.829039,-1568.367719,9899.542471,34.997526,-1561.199734,9891.906378,34.202,...,-1.072319,0.451414,-1.516815e-13,2.598795e-13,40.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
147,1.550081e+15,-3.417098,-10.179369,0.844909,-1568.475097,9899.587674,34.995304,-1561.456686,9891.450017,34.267,...,-1.072319,0.451414,-1.516815e-13,2.598795e-13,40.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
